In [2]:
install.packages("class",repos='"https://cran.r-project.org"')

Warning message:
"unable to access index for repository "https://cran.r-project.org"/src/contrib:
  scheme not supported in URL '"https://cran.r-project.org"/src/contrib/PACKAGES'"Warning message:
"package 'class' is not available (for R version 3.3.2)"Warning message:
"unable to access index for repository "https://cran.r-project.org"/bin/windows/contrib/3.3:
  scheme not supported in URL '"https://cran.r-project.org"/bin/windows/contrib/3.3/PACKAGES'"

In [37]:
library("class")

In [38]:
gcd<-read.csv("german_credit_data1.csv")

# cleaning the Data

## Removing all the NA,s in the data set

In [39]:
gcd<-na.omit(gcd)

## Changing all the Categorical attributes to numerical values for efficient processing of k-means algorithm.

In [40]:
gcd$Purpose <- as.numeric(factor(gcd$Purpose , levels=c("radio/TV",
 "furniture/equipment",
 "car",
 "business",
 "domestic appliances",
 "repairs",
 "vacation/others",
 "education"
)))

In [41]:
gcd$Housing <- as.numeric(factor(gcd$Housing , levels=c("rent" ,
          "free", "own")))
gcd$Sex <- as.numeric(factor(gcd$Sex , levels=c("male" ,
          "female")))
gcd$Checking.account <- as.numeric(factor(gcd$Checking.account , levels=c("little" ,
          "moderate", "rich")))
gcd$Saving.accounts <- as.numeric(factor(gcd$Saving.accounts , levels=c("little" ,
          "moderate", "quite rich","rich")))

# Normalizing the values of all the variables so that the values are in the range 0-1 so that calculating eucledian distance is done efficiently. 

In [42]:
gcd$Age<-(gcd$Age-min(gcd$Age))/(max(gcd$Age)-min(gcd$Age))
gcd$Sex<-(gcd$Sex-min(gcd$Sex))/(max(gcd$Sex)-min(gcd$Sex))
gcd$Job<-(gcd$Job-min(gcd$Job))/(max(gcd$Job)-min(gcd$Job))
gcd$Saving.accounts<-(gcd$Saving.accounts-min(gcd$Saving.accounts))/(max(gcd$Saving.accounts)-min(gcd$Saving.accounts))
gcd$Checking.account<-(gcd$Checking.account-min(gcd$Checking.account))/(max(gcd$Checking.account)-min(gcd$Checking.account))
gcd$Credit.amount<-(gcd$Credit.amount-min(gcd$Credit.amount))/(max(gcd$Credit.amount)-min(gcd$Credit.amount))
gcd$Duration<-(gcd$Duration-min(gcd$Duration))/(max(gcd$Duration)-min(gcd$Duration))
gcd$Purpose<-(gcd$Purpose-min(gcd$Purpose))/(max(gcd$Purpose)-min(gcd$Purpose))

In [43]:
head(gcd)

X. Age        Sex Job       Housing Saving.accounts Checking.account
2   1 0.05357143 1   0.6666667 3       0               0.5             
4   3 0.46428571 0   0.6666667 2       0               0.0             
5   4 0.60714286 0   0.6666667 2       0               0.0             
8   7 0.28571429 0   1.0000000 1       0               0.5             
10  9 0.16071429 0   1.0000000 3       0               0.5             
11 10 0.10714286 1   0.6666667 1       0               0.5             
   Credit.amount Duration   Purpose   Credit.Risks
2  0.31270663    0.63636364 0.0000000 2           
4  0.41910954    0.54545455 0.1428571 1           
5  0.25314084    0.27272727 0.2857143 2           
8  0.36764382    0.45454545 0.2857143 1           
10 0.27319815    0.36363636 0.2857143 2           
11 0.05614944    0.09090909 0.2857143 2

# Removing the first coloumn since it doesnt make any sense in clustering

In [44]:
gcd<-gcd[,-1]
head(gcd)

Age        Sex Job       Housing Saving.accounts Checking.account
2  0.05357143 1   0.6666667 3       0               0.5             
4  0.46428571 0   0.6666667 2       0               0.0             
5  0.60714286 0   0.6666667 2       0               0.0             
8  0.28571429 0   1.0000000 1       0               0.5             
10 0.16071429 0   1.0000000 3       0               0.5             
11 0.10714286 1   0.6666667 1       0               0.5             
   Credit.amount Duration   Purpose   Credit.Risks
2  0.31270663    0.63636364 0.0000000 2           
4  0.41910954    0.54545455 0.1428571 1           
5  0.25314084    0.27272727 0.2857143 2           
8  0.36764382    0.45454545 0.2857143 1           
10 0.27319815    0.36363636 0.2857143 2           
11 0.05614944    0.09090909 0.2857143 2

In [45]:
dim(gcd)

[1] 522  10

# Creating three different vectors to store misclassrate,samplingrate,clusters after each iteration and appending to it

In [46]:
misclassrate<-vector()
samplingrate<-vector()
clusters<-vector()

In [47]:
head(gcd)

Age        Sex Job       Housing Saving.accounts Checking.account
2  0.05357143 1   0.6666667 3       0               0.5             
4  0.46428571 0   0.6666667 2       0               0.0             
5  0.60714286 0   0.6666667 2       0               0.0             
8  0.28571429 0   1.0000000 1       0               0.5             
10 0.16071429 0   1.0000000 3       0               0.5             
11 0.10714286 1   0.6666667 1       0               0.5             
   Credit.amount Duration   Purpose   Credit.Risks
2  0.31270663    0.63636364 0.0000000 2           
4  0.41910954    0.54545455 0.1428571 1           
5  0.25314084    0.27272727 0.2857143 2           
8  0.36764382    0.45454545 0.2857143 1           
10 0.27319815    0.36363636 0.2857143 2           
11 0.05614944    0.09090909 0.2857143 2

In [48]:
sr<-0.8
for(sr in c(0.5,0.6,0.7,0.8,0.9)){
npoints<-dim(gcd)[1]
numtestl<-npoints*(1-sr)
training<-sample(1:npoints,npoints*sr,replace=FALSE)
#head(training)
train<-subset(gcd[training,],select=c(Age,Sex,Job,Housing,Saving.accounts,Checking.account,Credit.amount,Duration,Purpose))
#head(train)
testing<-setdiff(1:npoints,training)
test<-subset(gcd[testing,],select=c(Age,Sex,Job,Housing,Saving.accounts,Checking.account,Credit.amount,Duration,Purpose))
#head(test)
ltrain<-gcd$Credit.Risks[training]
ltest<-gcd$Credit.Risks[testing]
#length(unique(ltrain))
#knn(train,test,ltrain,k=3)
print(sr)
for(k in 1:20){
  #print(k)
  pred<-knn(train,test,ltrain,k)
  falsel<-sum(pred!=ltest)
  missrate<-falsel/numtestl
  print(c(k,missrate))
    misclassrate<-c(missrate,misclassrate)
    samplingrate<-c(samplingrate,sr)
    clusters<-c(clusters,k)
}
}

[1] 0.5
[1] 1.00000 0.48659
[1] 2.0000000 0.4367816
[1] 3.0000000 0.4789272
[1] 4.0000000 0.4482759
[1] 5.0000000 0.4789272
[1] 6.0000000 0.4942529
[1] 7.000000 0.467433
[1] 8.00000 0.48659
[1] 9.0000000 0.4559387
[1] 10.0000000  0.4636015
[1] 11.000000  0.440613
[1] 12.0000000  0.4252874
[1] 13.0000000  0.4444444
[1] 14.0000000  0.4252874
[1] 15.000000  0.440613
[1] 16.0000000  0.4329502
[1] 17.0000000  0.4252874
[1] 18.0000000  0.4252874
[1] 19.0000000  0.4061303
[1] 20.0000000  0.4291188
[1] 0.6
[1] 1.0000000 0.4454023
[1] 2.0000000 0.4454023
[1] 3.0000000 0.3975096
[1] 4.0000000 0.4166667
[1] 5.0000000 0.4310345
[1] 6.0000000 0.4022989
[1] 7.0000000 0.4214559
[1] 8.0000000 0.3975096
[1] 9.0000000 0.3783525
[1] 10.0000000  0.3783525
[1] 11.000000  0.387931
[1] 12.0000000  0.3975096
[1] 13.0000000  0.4022989
[1] 14.000000  0.387931
[1] 15.0000000  0.3831418
[1] 16.0000000  0.3687739
[1] 17.0000000  0.3735632
[1] 18.0000000  0.3783525
[1] 19.0000000  0.3927203
[1] 20.0000000  0.378352

# Finally storing all the above vectors in a dataframe and finding the least misclassification rate

In [49]:
tabb<-data.frame(samplingrate,clusters,misclassrate,misclassrate*100)

In [50]:
tabb

samplingrate clusters misclassrate misclassrate...100
1   0.5           1       0.2873563    28.73563          
2   0.5           2       0.2298851    22.98851          
3   0.5           3       0.2298851    22.98851          
4   0.5           4       0.2490421    24.90421          
5   0.5           5       0.2681992    26.81992          
6   0.5           6       0.2873563    28.73563          
7   0.5           7       0.3065134    30.65134          
8   0.5           8       0.3065134    30.65134          
9   0.5           9       0.2490421    24.90421          
10  0.5          10       0.2873563    28.73563          
11  0.5          11       0.2490421    24.90421          
12  0.5          12       0.3065134    30.65134          
13  0.5          13       0.3639847    36.39847          
14  0.5          14       0.2873563    28.73563          
15  0.5          15       0.2298851    22.98851          
16  0.5          16       0.3448276    34.48276          
17  0.5          17       0.2873563    28.73563          
18  0.5          18       0.4022989    40.22989          
19  0.5          19       0.5172414    51.72414          
20  0.5          20       0.4789272    47.89272          
21  0.6           1       0.3831418    38.31418          
22  0.6           2       0.3831418    38.31418          
23  0.6           3       0.3927203    39.27203          
24  0.6           4       0.3735632    37.35632          
25  0.6           5       0.3448276    34.48276          
26  0.6           6       0.3735632    37.35632          
27  0.6           7       0.4022989    40.22989          
28  0.6           8       0.3735632    37.35632          
29  0.6           9       0.3639847    36.39847          
30  0.6          10       0.3735632    37.35632          
... ...          ...      ...          ...               
71  0.8          11       0.3783525    37.83525          
72  0.8          12       0.3783525    37.83525          
73  0.8          13       0.3975096    39.75096          
74  0.8          14       0.4214559    42.14559          
75  0.8          15       0.4022989    40.22989          
76  0.8          16       0.4310345    43.10345          
77  0.8          17       0.4166667    41.66667          
78  0.8          18       0.3975096    39.75096          
79  0.8          19       0.4454023    44.54023          
80  0.8          20       0.4454023    44.54023          
81  0.9           1       0.4291188    42.91188          
82  0.9           2       0.4061303    40.61303          
83  0.9           3       0.4252874    42.52874          
84  0.9           4       0.4252874    42.52874          
85  0.9           5       0.4329502    43.29502          
86  0.9           6       0.4406130    44.06130          
87  0.9           7       0.4252874    42.52874          
88  0.9           8       0.4444444    44.44444          
89  0.9           9       0.4252874    42.52874          
90  0.9          10       0.4406130    44.06130          
91  0.9          11       0.4636015    46.36015          
92  0.9          12       0.4559387    45.59387          
93  0.9          13       0.4865900    48.65900          
94  0.9          14       0.4674330    46.74330          
95  0.9          15       0.4942529    49.42529          
96  0.9          16       0.4789272    47.89272          
97  0.9          17       0.4482759    44.82759          
98  0.9          18       0.4789272    47.89272          
99  0.9          19       0.4367816    43.67816          
100 0.9          20       0.4865900    48.65900

In [51]:
min(tabb$misclassrate)

[1] 0.2298851

In [52]:
min(tabb$misclassrate*100)

[1] 22.98851